In [12]:
#!pip install fhirclient
#!pip install fhirpy

In [13]:
import fhirpy

In [15]:
f = fhirpy.SyncFHIRClient('https://sjb2dicomcast-fhir.azurewebsites.net')

In [21]:
f.resources('Patient').fetch_all()

[<SyncFHIRResource Patient/3fc7ec5e-70ee-4f00-b270-921a043dfc2d>,
 <SyncFHIRResource Patient/caf9951f-9e86-403b-80ff-49503398eb24>,
 <SyncFHIRResource Patient/59aafb43-dcac-460e-8637-c6f2f92f738a>,
 <SyncFHIRResource Patient/f1ab468f-712d-42d2-b47c-ad40112a2de2>,
 <SyncFHIRResource Patient/d6f030fa-a825-43eb-ba13-3b7667e0ba57>,
 <SyncFHIRResource Patient/b3c36c31-7872-4437-bfe2-6c1fe0d896dc>,
 <SyncFHIRResource Patient/af834337-f9f7-449c-ad3a-481db3ff7749>,
 <SyncFHIRResource Patient/962eec9d-fe70-4b62-8158-128324bf0178>]

In [ ]:
f.resources('Patient').

In [ ]:
import json
import os
import requests


#fhir server endpoint
URL = "http://localhost:8080/hapi-fhir-jpaserver/fhir/"

#fhir server json header content
headers = {"Content-Type": "application/fhir+json;charset=utf-8"}

#loop over all files in the output folder in order to upload each json file for each patient.
for dirpath, dirnames, files in os.walk('/Users/rajvansia/Documents/GitHub/synthea/output/fhir'):

    for file_name in files:
        with open('/Users/rajvansia/Documents/GitHub/synthea/output/fhir/'+file_name, "r") as bundle_file:
                data = bundle_file.read()

        r = requests.post(url = URL, data = data, headers = headers)

        #output file name that was processed
        print(file_name)

In [3]:
from fhirclient import client
settings = {
    'app_id': 'sjbdicom',
    'api_base': 'https://sjb2dicomcast-fhir.azurewebsites.net'
}
smart = client.FHIRClient(settings=settings)

import fhirclient.models.patient as p
#patient = p.Patient.read('hca-pat-1', smart.server)
#patient.birthDate.isostring
# '1963-06-12'
#smart.human_name(patient.name[0])
# 'Christy Ebert'

In [10]:
smart.prepare()

FHIRValidationError: {root}:
  rest.0:
    operation.0:
      definition:
        `with_json()` on <class 'models.fhirreference.FHIRReference'> only takes dict or list of dict, but you provided <class 'str'>
      'Non-optional property "definition" on <models.capabilitystatement.CapabilityStatementRestOperation object at 0x000001A87675A148> is missing'
    resource.0:
      profile:
        `with_json()` on <class 'models.fhirreference.FHIRReference'> only takes dict or list of dict, but you provided <class 'str'>
  'Non-optional property "date" on <models.capabilitystatement.CapabilityStatement object at 0x000001A876971B48> is missing'
  'Non-optional property "acceptUnknown" on <models.capabilitystatement.CapabilityStatement object at 0x000001A876971B48> is missing'

In [9]:
p.Patient.read(, smart.server)


FHIRNotFoundException: <Response [404]>

In [22]:
import os
import requests
import io
import pandas as pd
import pydicom
from pathlib import Path
import time

from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue

#see https://stackoverflow.com/questions/15746558/how-to-send-a-multipart-related-with-requests-in-python

#from fastcore.foundation import * #Patch here (but also imports fastcore.imports (which is not sufficient))
    # But parallel is NOT here... :-(

from fastai2.data.all import *
#from fastai2 import *



In [41]:

dicom_server_name = "sjbdcast6-dicom"
url = f"https://{dicom_server_name}.azurewebsites.net/studies"
url

'https://sjbdcast6-dicom.azurewebsites.net/studies'

In [24]:
# Cell
@patch
def dcmread(fn:Path, force = False): #, force = False):
    "Open a `DICOM` file"
    return pydicom.dcmread(str(fn), force)

# Cell
def _cast_dicom_special(x):
    cls = type(x)
    if not cls.__module__.startswith('pydicom'): return x
    if cls.__base__ == object: return x
    return cls.__base__(x)

def _split_elem(res,k,v):
    if not isinstance(v,DcmMultiValue): return
    res[f'Multi{k}'] = 1
    for i,o in enumerate(v): res[f'{k}{"" if i==0 else i}']=o

        # Cell
@patch
def as_dict(self:DcmDataset, px_summ=False): #, window=dicom_windows.brain):
    pxdata = (0x7fe0,0x0010)
    vals = [self[o] for o in self.keys() if o != pxdata]
    its = [(v.keyword,v.value) for v in vals]
    res = dict(its)
    res['fname'] = self.filename
    for k,v in its: _split_elem(res,k,v)
    if not px_summ: return res
    stats = 'min','max','mean','std'
#    try:
#        pxs = self.pixel_array
#        for f in stats: res['img_'+f] = getattr(pxs,f)()
#        res['img_pct_window'] = self.pct_in_window(*window)
#    except Exception as e:
#        for f in stats: res['img_'+f] = 0
#        print(res,e)
    for k in res: res[k] = _cast_dicom_special(res[k])
    return res

# Cell
def _dcm2dict(fn, **kwargs): return fn.dcmread().as_dict(**kwargs)

# Cell
@delegates(parallel)
def _from_dicoms(cls, fns, n_workers=0, **kwargs):
    return pd.DataFrame(parallel(_dcm2dict, fns, n_workers=n_workers, **kwargs))
pd.DataFrame.from_dicoms = classmethod(_from_dicoms)

In [25]:

# Validate goes beyond checking the dicom header, but attempts a actually load every single file into a Pydicom Dataset
#    This later step has shown effective for ensuring that the metadata can be loaded into a dataset
def get_dicom_files_from_directory(dirpath,extension="*.dcm", validate=False):
    path = Path(dirpath) 
    # pydicom 2 has an is_valid method to replace is_dicom (maybe look at pydicom2 for other reasons)
    files = [x for x in path.rglob(extension)] # if pydicom.misc.is_dicom(str(x))] # checks for DICM in header (fast and loose)
    if validate:
        for file in files[:]:
            #pydicom.validate() #is this only in pydicom 2?
            try:
                #print(file)
                x = dcmread(file)           
            except:
                # remove file if it throws an exception
                files.remove(x)
    #Potentially look at pydicom.dataset.validate_file_meta(file_meta, enforce_standard=True) for deeper validation
    #  https://pydicom.github.io/pydicom/dev/reference/generated/pydicom.dataset.validate_file_meta.html
    return files
    

In [45]:
#files = get_dicom_files_from_directory('C:\\githealth\\hackathon-dataset\\hackathon-images', validate=False) # all succeed
#files = get_dicom_files_from_directory('C:\\githealth\\dicom-samples\\rubomedica.com') #all succeed
#files = get_dicom_files_from_directory('C:\\githealth\\dicom-samples\\barre.dev') #409 and 500 errors on first upload
#files = get_dicom_files_from_directory('C:\\githealth\\dicom-samples\\dicomlibrary.com\\series-000001') #409 and 500 errors on first upload
#files = get_dicom_files_from_directory('C:\\!!delete4DLung', validate=False)
#files = get_dicom_files_from_directory('C:\\!DicomDataTCIA\\4D-Lung', validate=False)
files = get_dicom_files_from_directory('C:\\githealth\\elastic-cast-reduced2', validate=False)
#files = get_dicom_files_from_directory('C:\\!!delete4DLung', validate=False)
#files = get_dicom_files_from_directory('C:\\!!delete4DLung', validate=False)

print(len(files))
                               

6189


In [46]:
files = [x for x in files if not 'pydicom' in str(x)]
print(len(files))

6189


In [28]:

# Can just use this to load all the files in a directory, instead of calling above
#dirname = 'C:\\githealth\\dicom-samples\\visus.com'
#files = [x for x in Path(dirname).rglob('*.dcm')] # if not 'pydicom' in str(x)] # exclude pydicom test files



In [29]:
# Stupid simple function to get a Dataframe, but can be extended later to add logic
def get_dicom_metadata_as_dataframe(list_of_files,columns=[],save_as_filename=''):
    df = pd.DataFrame.from_dicoms(files)
    if save_as_filename != '':
        df.to_csv(save_as_filename)
    return df
        
    



In [30]:
## The following goes and gets metadata for 4d-lung


In [31]:
len(files)

1108

In [32]:
#df = pd.DataFrame.from_dicoms(files)
#df.to_csv('TestDicom2.csv')

In [28]:
df = get_dicom_metadata_as_dataframe(files)

In [29]:
df.to_csv('4D_lung_metadata.csv')

In [33]:
from urllib3.filepost import encode_multipart_formdata, choose_boundary

def encode_multipart_related(fields, boundary=None):
    if boundary is None:
        boundary = choose_boundary()

    body, _ = encode_multipart_formdata(fields, boundary)
    content_type = str('multipart/related; boundary=%s' % boundary)

    return body, content_type

In [34]:
df.head()

NameError: name 'df' is not defined

In [35]:
def upload_single_dcm_file(server_url,filepath):
    with open(filepath,'rb') as reader:
        rawfile = reader.read()
    files = {'file': ('dicomfile', rawfile, 'application/dicom')}

    #encode as multipart_related
    body, content_type = encode_multipart_related(fields = files)
    
    headers = {'Accept':'application/dicom+json', "Content-Type":content_type}

    response = requests.post(url, body, headers=headers) #, verify=False)
    
    #return the response object to allow for further processing
    
    #example usage
    #r = upload_single_dcm_file(url,'C:\\githealth\\dicom-samples\\visus.com\\case4\\case4a_002.dcm')
    #print(r.status_code)
    #print(r.request.headers)
    
    return response

In [36]:
#r = upload_dcm_file(url,'C:\\githealth\\dicom-samples\\visus.com\\case4\\case4a_002.dcm')
#print(r.status_code)
#print(r.request.headers)

In [37]:
from collections import OrderedDict
def store_files_to_dicomweb_with_logging_to_dataframe(filepaths, df = None):
    if df is None:
        # Then we have a blank dataframe, maybe do something here.
        #   for now, I'm just using a blank one.
        #columns = ['fname','status_code','response_notes']
        df = pd.DataFrame()
        df['fname']=filepaths
    
    # Fastest to just use lists and dictionaries then create a new dataframe from that...
    numfiles=len(filepaths)
    current_count=0
    
    files_info = []
    for file in filepaths:
        
        # Upload a single file at a time (and time it)
        r = upload_single_dcm_file(url,file)     # call API and get response 
        d = OrderedDict()   # create a new ordered dictionary
        
        # Add the information to the dictionary
        d['fname'] = str(file)     
        d['method'] = r.request.method
        d['url'] = r.url
        d['path_url'] = r.request.path_url
        d['request_headers'] = str(r.request.headers)
        d['request_body_trimmed'] = r.request.body[0:150]   

        d['ok'] = r.ok
        d['status_code'] = r.status_code  
        d['reason'] = r.reason
        d['response_headers'] = r.headers
        d['response_text'] = r.text  
        d['elapsed_time'] = r.elapsed
        d['apparent_encoding'] = r.apparent_encoding
        d['encoding'] = r.encoding
        
 
        files_info.append(d)
        current_count += 1
        if current_count%250==0:
            print(f"{current_count} of {numfiles} uploaded")

    # Create a dataframe from the uploads
    df = pd.DataFrame(files_info)               
    return df
        
    
    

In [47]:
df = store_files_to_dicomweb_with_logging_to_dataframe(files)

250 of 6189 uploaded
500 of 6189 uploaded
750 of 6189 uploaded
1000 of 6189 uploaded
1250 of 6189 uploaded
1500 of 6189 uploaded
1750 of 6189 uploaded
2000 of 6189 uploaded
2250 of 6189 uploaded
2500 of 6189 uploaded
2750 of 6189 uploaded
3000 of 6189 uploaded
3250 of 6189 uploaded
3500 of 6189 uploaded
3750 of 6189 uploaded
4000 of 6189 uploaded
4250 of 6189 uploaded
4500 of 6189 uploaded
4750 of 6189 uploaded
5000 of 6189 uploaded
5250 of 6189 uploaded
5500 of 6189 uploaded
5750 of 6189 uploaded
6000 of 6189 uploaded


In [48]:
df.head(15)

,fname,method,url,path_url,request_headers,request_body_trimmed,ok,status_code,reason,response_headers,response_text,elapsed_time,apparent_encoding,encoding
0,C:\githealth\elastic-cast-reduced2\Andy SIIM\Ct_Chest_NC - 0\HELICAL_MODE_2\IM-0026-0001.dcm,POST,https://sjbdcast6-dicom.azurewebsites.net/studies,/studies,"{'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/dicom+json', 'Connection': 'keep-alive', 'Content-Type': 'multipart/related; boundary=ce5a66c39ff369370893f05d297baf6f', 'Content-Length': '528848'}","b'--ce5a66c39ff369370893f05d297baf6f\r\nContent-Disposition: form-data; name=""file""; filename=""dicomfile""\r\nContent-Type: application/dicom\r\n\r\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'",True,200,OK,"[Content-Length, Content-Type, Server, Request-Context, X-Content-Type-Options, X-Powered-By, Date]","{""00081199"":{""vr"":""SQ"",""Value"":[{""00081150"":{""vr"":""UI"",""Value"":[""1.2.840.10008.5.1.4.1.1.2""]},""00081155"":{""vr"":""UI"",""Value"":[""1.3.6.1.4.1.14519.5.2.1.6450.9002.560831737608650202608209404565""]},""00081190"":{""vr"":""UR"",""Value"":[""https://sjbdcast6-dicom.azurewebsites.net/studies/1.3.6.1.4.1.14519.5.2.1.6450.9002.339372202011004731829363267423/series/1.3.6.1.4.1.14519.5.2.1.6450.9002.747485993156290148171122461766/instances/1.3.6.1.4.1.14519.5.2.1.6450.9002.560831737608650202608209404565""]}}]}}",00:00:00.462254,ascii,utf-8
1,C:\githealth\elastic-cast-reduced2\Andy SIIM\Ct_Chest_NC - 0\HELICAL_MODE_2\IM-0026-0002.dcm,POST,https://sjbdcast6-dicom.azurewebsites.net/studies,/studies,"{'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/dicom+json', 'Connection': 'keep-alive', 'Content-Type': 'multipart/related; boundary=2003d0802211c04f37575fb168a88f88', 'Content-Length': '528848'}","b'--2003d0802211c04f37575fb168a88f88\r\nContent-Disposition: form-data; name=""file""; filename=""dicomfile""\r\nContent-Type: application/dicom\r\n\r\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'",True,200,OK,"[Content-Length, Content-Type, Server, Request-Context, X-Content-Type-Options, X-Powered-By, Date]","{""00081199"":{""vr"":""SQ"",""Value"":[{""00081150"":{""vr"":""UI"",""Value"":[""1.2.840.10008.5.1.4.1.1.2""]},""00081155"":{""vr"":""UI"",""Value"":[""1.3.6.1.4.1.14519.5.2.1.6450.9002.297826588662579079841363899995""]},""00081190"":{""vr"":""UR"",""Value"":[""https://sjbdcast6-dicom.azurewebsites.net/studies/1.3.6.1.4.1.14519.5.2.1.6450.9002.339372202011004731829363267423/series/1.3.6.1.4.1.14519.5.2.1.6450.9002.747485993156290148171122461766/instances/1.3.6.1.4.1.14519.5.2.1.6450.9002.297826588662579079841363899995""]}}]}}",00:00:00.161432,ascii,utf-8
2,C:\githealth\elastic-cast-reduced2\Andy SIIM\Ct_Chest_NC - 0\HELICAL_MODE_2\IM-0026-0003.dcm,POST,https://sjbdcast6-dicom.azurewebsites.net/studies,/studies,"{'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/dicom+json', 'Connection': 'keep-alive', 'Content-Type': 'multipart/related; boundary=65acc087fc7a980dd5cf26cd2e791169', 'Content-Length': '528848'}","b'--65acc087fc7a980dd5cf26cd2e791169\r\nContent-Disposition: form-data; name=""file""; filename=""dicomfile""\r\nContent-Type: application/dicom\r\n\r\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'",True,200,OK,"[Content-Length, Content-Type, Server, Request-Context, X-Content-Type-Options, X-Powered-By, Date]","{""00081199"":{""vr"":""SQ"",""Value"":[{""00081150"":{""vr"":""UI"",""Value"":[""1.2.840.10008.5.1.4.1.1.2""]},""00081155"":{""vr"":""UI"",""Value"":[""1.3.6.1.4.1.14519.5.2.1.6450.9002.225319844260836053102353216326""]},""00081190"":{""vr"":""UR"",""Value"":[""https://sjbdcast6-dicom.azurewebsites.net/studies/1.3.6.1.4.1.14519.5.2.1.6450.9002.339372202011004731829363267423/series/1.3.6.1.4.1.14519.5.2.1.6450.9002.747485993156290148171122461766/instances/1.3.6.1.4.1.14519.5.2.1.6450.9002.225319844260836053102353216326""]}}]}}

In [49]:
df.to_csv('SIIM_data_Nov18.csv')

In [54]:
len(df)

10

In [55]:
# The following is for chunked upload of larger directories


In [ ]:
len(files)

In [ ]:
def split_list(a_list, size):
     lists = []
     while len(a_list) > size:
         pice = a_list[:size]
         lists.append(pice)
         a_list = a_list[size:]
     lists.append(a_list)
     return lists

In [ ]:
def process_file_subset(file_subset):
    #print(f'Processing {len(file_subset)} files')
    #df_meta = pd.DataFrame()
    #df_upload = pd.DataFrame() 
    df_meta = get_dicom_metadata_as_dataframe(file_subset)
    #df_meta.to_csv('c:\\!mlhack\\metadata-mlhack_train_subset.csv')
    df_upload = store_files_to_dicomweb_with_logging_to_dataframe(file_subset)
    #df_upload.to_csv('c:\\!mlhack\\upload-data-mlhack_train_subset.csv')
    return df_meta, df_upload

In [ ]:
# Process all the files in chunks
batch_size = 250
df_meta = pd.DataFrame()
df_upload = pd.DataFrame()

groups_of_files = split_list(files,batch_size)

for i,group in enumerate(groups_of_files):
    print(f'Processing {i} of {len(groups_of_files)} groups with {len(group)} files each')
    df_meta_subset, df_upload_subset = process_file_subset(group)
    df_meta = pd.concat([df_meta,df_meta_subset])
    df_upload = pd.concat([df_upload,df_upload_subset])
    df_meta.to_csv('c:\\!mlhack\\metadata-mlhack_train_sofar.csv')
    df_upload.to_csv('c:\\!mlhack\\upload-data-mlhack_train_sofar.csv')

In [ ]:
df_meta.to_csv('c:\\!mlhack\\metadata-mlhack_train.csv')
df_upload.to_csv('c:\\!mlhack\\upload-data-mlhack_train.csv')